# Minimal FaceGreeter Flask with NGROK

This notebook tests a basic Flask server in Google Colab with ngrok to debug connection issues. It runs a simple Flask app with a `/hello` endpoint, exposes it via ngrok, and monitors for kernel crashes. If stable, we'll add git clone, `.env`, and video processing.

## Prerequisites
- Google Colab environment with internet access.
- `NGROK_AUTHTOKEN` stored in Colab Secrets.

## Steps
1. Install Flask and pyngrok.
2. Create and run a basic Flask app.
3. Expose the app via ngrok.
4. Monitor resources and test the `/hello` endpoint.

**Goal**: Confirm Flask runs and is accessible via ngrok.

In [ ]:
# Install Flask, pyngrok, and monitor resources
import os
import psutil
from google.colab import userdata

# Log initial resource usage
print(f'Initial RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Initial CPU: {psutil.cpu_percent()}%')

# Check NGROK_AUTHTOKEN
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')

# Purge pip cache
!pip cache purge

# Install Flask and pyngrok
!pip install --no-cache-dir flask==3.0.3 pyngrok==7.1.6

# Verify installed packages
!pip list | grep -E 'flask|pyngrok'

# Log resource usage after install
print(f'Post-install RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-install CPU: {psutil.cpu_percent()}%')

print('Flask and pyngrok installed. Proceed to the next cell to run the Flask app with ngrok.')

In [ ]:
# Run Flask with ngrok
from flask import Flask, jsonify
from pyngrok import ngrok
import os
import psutil
import threading
import time
from google.colab import userdata

# Log resource usage before Flask
print(f'Pre-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Pre-Flask CPU: {psutil.cpu_percent()}%')

# Get NGROK_AUTHTOKEN
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
if not ngrok_token:
    raise ValueError('NGROK_AUTHTOKEN not found in Colab Secrets.')

# Set ngrok auth token
ngrok.set_auth_token(ngrok_token)

# Create Flask app
app = Flask(__name__)

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify({"success": True, "message": "Hello from Flask!"})

# Run Flask in a separate thread
def run_flask():
    print('Starting Flask server on http://0.0.0.0:8000')
    app.run(host='0.0.0.0', port=8000, debug=False, use_reloader=False)

print('Starting Flask server...')
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)

# Start ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f'Flask API available at: {public_url}')
except Exception as e:
    print(f'Ngrok failed to start: {str(e)}')
    os.system('pkill python')
    os.system('fuser -k 8000/tcp')
    raise

# Check running processes and port
os.system('ps aux | grep python')
os.system('netstat -tuln | grep 8000')

# Log resource usage after Flask
print(f'Post-Flask RAM: {psutil.virtual_memory().used / 1024**3:.2f} GB / {psutil.virtual_memory().total / 1024**3:.2f} GB')
print(f'Post-Flask CPU: {psutil.cpu_percent()}%')

# Keep the notebook running
print(f'Flask server running. Test the API at {public_url}/hello.')
print('Press Ctrl+C in this cell to stop the Flask server.')
try:
    flask_thread.join()
except KeyboardInterrupt:
    print('Stopping Flask server...')
    os.system('pkill python')
    os.system('pkill ngrok')
    os.system('fuser -k 8000/tcp')